In [1]:
from utils import get_pretrained_vocab
from modules import BERT
import torch

In [2]:
tokenizer = get_pretrained_vocab('vocab_file/tokenizer_syllable_large.json')
tagger = get_pretrained_vocab('vocab_file/tagger.json')

In [3]:
print(tokenizer.vocab_size)
print(tagger.vocab_size)

4751
22


In [13]:
BEST_MODEL_PARAMS_PATH = "model_weights/best_model_params_syllable_large.pt"
# Define model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = BERT(
    num_classes=tagger.vocab_size, 
    clf_hidden_dim=1000,
    vocab_size=tokenizer.vocab_size,
    pad_token_id=tokenizer.pad_token_id,
    num_encoder_blocks=12, 
    num_heads=16,
    embed_dim=1024,
    qkv_bias=True,
    ff_dim=4096,
    ff_activate_fn='gelu',
    ).to(device)
model.load_state_dict(torch.load(BEST_MODEL_PARAMS_PATH))
model.eval()

BERT(
  (encoder): TransformerEncoder(
    (embedding): Embedding(4751, 1024, padding_idx=0)
    (positional_encoding): SinusoidEncoding()
    (dropout): Dropout(p=0.1, inplace=False)
    (encoder_blocks): ModuleList(
      (0): TransformerEncoderBlock(
        (self_mha): MultiHeadAttention(
          (qkv_proj): Linear(in_features=64, out_features=192, bias=True)
          (fc_out): Linear(in_features=1024, out_features=1024, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): FeedForwardLayer(
          (linear1): Linear(in_features=1024, out_features=4096, bias=True)
          (linear2): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activate_fn): GELU()
        )
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
        (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((1024,), e

In [45]:
text = 'Bộ Y tế cho biết ngày 13/1 có 55 ca mắc COVID-19 , giảm nhẹ so với ngày trước đó; Trong ngày có gần 400 bệnh nhân khỏi, đây cũng là ngày có số bệnh nhân khỏi nhiều nhất kể từ đầu tháng 1/2023 đến nay.'

In [46]:
tokens = tokenizer.tokenize(text, return_dict=True, do_clean=True)

In [47]:
tokens

[('bộ', 391),
 ('y', 88),
 ('tế', 89),
 ('cho', 180),
 ('biết', 195),
 ('ngày', 15),
 ('13', 4746),
 ('/', 780),
 ('1', 4746),
 ('', 4749),
 ('có', 42),
 ('55', 4746),
 ('', 4749),
 ('ca', 3),
 ('mắc', 668),
 ('covid', 30),
 ('91', 4746),
 ('-', 31),
 (',', 38),
 ('', 4749),
 ('giảm', 697),
 ('nhẹ', 210),
 ('so', 1726),
 ('với', 55),
 ('ngày', 15),
 ('trước', 174),
 ('đó', 78),
 (';', 111),
 ('trong', 39),
 ('ngày', 15),
 ('có', 42),
 ('gần', 334),
 ('400', 4746),
 ('', 4749),
 ('bệnh', 19),
 ('nhân', 20),
 ('khỏi', 29),
 (',', 38),
 ('đây', 338),
 ('cũng', 607),
 ('là', 17),
 ('ngày', 15),
 ('có', 42),
 ('số', 40),
 ('bệnh', 19),
 ('nhân', 20),
 ('khỏi', 29),
 ('nhiều', 372),
 ('nhất', 74),
 ('kể', 936),
 ('từ', 34),
 ('đầu', 447),
 ('tháng', 247),
 ('1', 4746),
 ('/', 780),
 ('2023', 4746),
 ('', 4749),
 ('đến', 36),
 ('nay', 186),
 ('.', 24)]

In [8]:
import string
def clean_word(word: str, special_sep='"&\'()*+-;?'):
    if word == '': return word
    integers = []
    punctuation = list(string.punctuation) 
    prev = []
    while len(word) > 0:
        if word[0] in punctuation:
            prev.append(word[0])
            word = word[1:]
        elif word[0] in integers:
            if prev != [] and prev[-1][-1] in integers:
                prev[-1] += word[0]
            else:
                prev.append(word[0])
            word = word[1:]
        else: break

    post = []
    while len(word) > 0:
        if word[-1] in punctuation:
            post.append(word[-1])
            word = word[:-1]
        elif word[-1] in integers:
            if post != [] and post[-1][-1] in integers:
                post[-1] += word[-1]
            else:
                post.append(word[-1])
            word = word[:-1]
        else: break

    return prev + [word] + post

In [48]:
results = []
for _ in text.split():
    results += clean_word(_)    
print(results)

['Bộ', 'Y', 'tế', 'cho', 'biết', 'ngày', '13/1', 'có', '55', 'ca', 'mắc', 'COVID-19', ',', '', 'giảm', 'nhẹ', 'so', 'với', 'ngày', 'trước', 'đó', ';', 'Trong', 'ngày', 'có', 'gần', '400', 'bệnh', 'nhân', 'khỏi', ',', 'đây', 'cũng', 'là', 'ngày', 'có', 'số', 'bệnh', 'nhân', 'khỏi', 'nhiều', 'nhất', 'kể', 'từ', 'đầu', 'tháng', '1/2023', 'đến', 'nay', '.']


In [49]:
input_ids = [tokenizer.stoi2(_) for _ in results]

In [50]:
input_ids = torch.tensor([input_ids], device=device)
attn_mask = torch.ones_like(input_ids, device=device)

In [51]:
input_ids.size()

torch.Size([1, 50])

In [52]:
output = model(input_ids, attn_mask, apply_tanh=True)

In [53]:
output = output.argmax(dim=-1)

In [54]:
tags = tagger.detokenize(output.tolist()[0])

In [55]:
for k, v in zip(results, tags):
    print(k, v)

Bộ O
Y O
tế O
cho O
biết O
ngày O
13/1 B-DATE
có O
55 O
ca O
mắc O
COVID-19 O
, O
 B-NAME
giảm O
nhẹ O
so O
với O
ngày O
trước O
đó O
; O
Trong O
ngày O
có O
gần O
400 O
bệnh O
nhân O
khỏi O
, O
đây O
cũng O
là O
ngày O
có O
số O
bệnh O
nhân O
khỏi O
nhiều O
nhất O
kể O
từ O
đầu O
tháng O
1/2023 O
đến O
nay O
. O
